In [7]:
import pandas as pd
import math
import numpy as np

# load data
parts = pd.read_csv('../Raw_Data/parts.csv')
sets = pd.read_csv('../Raw_Data/sets.csv')
colors = pd.read_csv('../Raw_Data/colors.csv')
inventories = pd.read_csv('../Raw_Data/inventories.csv')
inventory_parts = pd.read_csv('../Raw_Data/inventory_parts.csv')
minifigs = pd.read_csv('../Raw_Data/minifigs.csv')
inventory_sets = pd.read_csv('../Raw_Data/inventory_sets.csv')
inventory_minifigs = pd.read_csv('../Raw_Data/inventory_minifigs.csv')
minifigures = pd.read_csv('../Raw_Data/minifigs.csv')
categories = pd.read_csv('../Raw_Data/part_categories.csv')
themes = pd.read_csv('../Raw_Data/themes.csv')


In [8]:
##### Find the final parent of a theme and make a dict ####

final_parent = {}

parent_dict = dict(zip(themes['id'], themes['parent_id']))

def find_final_parent(id):
    parent = parent_dict[id]
    if math.isnan(parent):
        return int(id), themes.loc[themes['id'] == id, 'name'].iloc[0]
    else:
        return find_final_parent(parent)
    
final_parent_dict = {id: find_final_parent(id) for id in themes['id']}

print(final_parent_dict)

{1: (1, 'Technic'), 3: (1, 'Technic'), 4: (1, 'Technic'), 16: (1, 'Technic'), 17: (1, 'Technic'), 18: (1, 'Technic'), 19: (1, 'Technic'), 20: (1, 'Technic'), 21: (1, 'Technic'), 22: (22, 'Creator'), 23: (22, 'Creator'), 34: (34, 'Make & Create'), 35: (34, 'Make & Create'), 50: (50, 'Town'), 51: (50, 'Town'), 52: (52, 'City'), 53: (52, 'City'), 54: (52, 'City'), 55: (52, 'City'), 56: (52, 'City'), 57: (52, 'City'), 58: (52, 'City'), 59: (52, 'City'), 60: (52, 'City'), 61: (52, 'City'), 63: (52, 'City'), 64: (52, 'City'), 65: (52, 'City'), 66: (52, 'City'), 67: (50, 'Town'), 86: (50, 'Town'), 87: (50, 'Town'), 88: (50, 'Town'), 89: (50, 'Town'), 90: (50, 'Town'), 91: (50, 'Town'), 92: (50, 'Town'), 93: (50, 'Town'), 94: (50, 'Town'), 104: (50, 'Town'), 105: (50, 'Town'), 106: (50, 'Town'), 112: (112, 'Racers'), 114: (112, 'Racers'), 118: (112, 'Racers'), 122: (112, 'Racers'), 123: (112, 'Racers'), 126: (126, 'Space'), 127: (126, 'Space'), 128: (126, 'Space'), 129: (126, 'Space'), 130: (1

In [9]:
##### Merge data  #####

# merge left side of data structure
# merge categories with parts
categories_with_parts = pd.merge(categories, parts, left_on='id', right_on='part_cat_id')
categories_with_parts.rename(columns={'name_x': 'category_name', 'name_y': 'part_name', 'id': 'category_id'}, inplace=True)
categories_with_parts.drop('part_cat_id', axis=1, inplace=True)

# merge categories and parts with inventory_parts
categories_with_parts_with_inventory_parts = pd.merge(categories_with_parts, inventory_parts, left_on='part_num', right_on='part_num')
categories_with_parts_with_inventory_parts.drop('img_url', axis=1, inplace=True)

# merge inventory_parts with colors
categories_with_parts_with_colors_inventory_parts = pd.merge(categories_with_parts_with_inventory_parts, colors, left_on='color_id', right_on='id')
categories_with_parts_with_colors_inventory_parts.drop('color_id', axis=1, inplace=True)
categories_with_parts_with_colors_inventory_parts.rename(columns={'id': 'color_id', 'name': 'color_name', 'quantity': 'part_quantity'}, inplace=True)


# merge middle of data structure
# merge minifgures with inventory_minifgures
minifigures_with_inventory_minifgures = pd.merge(minifigs, inventory_minifigs, left_on='fig_num', right_on='fig_num')
minifigures_with_inventory_minifgures.rename(columns={'name': 'minifig_name', 'quantity': 'minifig_quantitiy'}, inplace=True)
minifigures_with_inventory_minifgures.drop('img_url', axis=1, inplace=True)

# merge right side of data structure
# merge sets with themes
sets_with_themes = pd.merge(sets, themes, left_on='theme_id', right_on='id')
sets_with_themes.drop(['img_url','theme_id'], axis=1, inplace=True)
sets_with_themes.rename(columns={'name_x': 'set_name', 'id': 'theme_id', 'name_y': 'theme_name'}, inplace=True)

for index, row in sets_with_themes.iterrows():                                      # just use main themes
    row_theme = row['theme_id']
    sets_with_themes.at[index, 'theme_id'] =  final_parent_dict[row_theme][0]
    sets_with_themes.at[index, 'theme_name'] =  final_parent_dict[row_theme][1]

sets_with_themes.drop('parent_id', axis=1, inplace=True)

#sets_with_themes = sets_with_themes[sets_with_themes.num_parts != 0]                  # delete all things that has no Lego parts
theme_names = sets_with_themes['theme_name'].unique()                                  # print themes
for name in theme_names:
    print(name)
print(len(themes))


"""
# merge sets and themes with inventory_sets
sets_with_themes_inventory_sets = pd.merge(sets_with_themes, inventory_sets, left_on='set_num', right_on='set_num')
sets_with_themes_inventory_sets.rename(columns={'quantity': 'set_quantity'}, inplace=True)
sets_with_themes_inventory_sets.info()
sets_with_themes_inventory_sets.to_csv('Merged_Data/inventory_sets.csv', index=False)
"""

# merge all together
sets_with_inventory = pd.merge(sets_with_themes, inventories, left_on='set_num', right_on='set_num')

parts_with_inventory = pd.merge(categories_with_parts_with_colors_inventory_parts, sets_with_inventory, left_on='inventory_id', right_on='id')
parts_with_inventory.drop('id', axis=1, inplace=True)
parts_with_inventory.rename(columns={'num_parts': 'number_of_parts'}, inplace=True)

minifigs_with_inventory = pd.merge(minifigures_with_inventory_minifgures, sets_with_inventory, left_on='inventory_id', right_on='id')
minifigs_with_inventory.drop('id', axis=1, inplace=True)
minifigs_with_inventory.rename(columns={'num_parts_x': 'number_of_minifig_parts', 'num_parts_y': 'number_of_parts'}, inplace=True)


# filter data for the full years
data_parts = parts_with_inventory[parts_with_inventory['year'] <= 2023]
data_minifigs = minifigs_with_inventory[minifigs_with_inventory['year'] <= 2023]


# save full data 
data_minifigs.to_csv('../Prepared_Data/data_minifigs.csv', index=False)
data_parts.to_csv('../Prepared_Data/data_parts.csv', index=False)

Technic
Town
Castle
Space
Books
System
Duplo
Icons
Service Packs
Creator
Universal Building Set
Gear
Educational and Dacta
Train
Bulk Bricks
Star Wars
Hobby Sets
Bionicle
Discovery
Pirates
Seasonal
Super Heroes Marvel
Clikits
LEGO Brand Store
Sports
Harry Potter
Model Team
City
Other
Modular Buildings
Factory
Monster Fighters
Legoland
The Hobbit and Lord of the Rings
Adventurers
Mindstorms
Classic
Promotional
Make & Create
Juniors
Toy Story
Disney
Gabby’s Dollhouse
Super Heroes DC
Aquazone
Ninja
Ninjago
Minitalia
Primo
LEGO Originals
Hero Factory
Racers
Fabuland
Jurassic World
Rock Raiders
Studios
Alpha Team
4 Juniors
Monkie Kid
Value Packs
Building Set with People
Boat
Freestyle
Time Cruisers
BrickLink Designer Program
Architecture
Indiana Jones
FIRST LEGO League
Atlantis
Prince of Persia
LEGO Exclusive
Master Builder Academy
LEGO Ideas and CUUSOO
Minecraft
Fusion
LEGO Art
Homemaker
Elves
Belville
Nexo Knights
Western
Universe
Games
Pharaoh's Quest
Friends
Disney Princess
Cars
Pirates

In [10]:
#### Merge data to final data structure to work with #####

# find relevant information
grouped_years = data_parts[['set_num','year']].drop_duplicates(subset='set_num')
grouped_themes = data_parts[['set_num','theme_id', 'theme_name']].drop_duplicates(subset='set_num')
grouped_parts = data_parts[['set_num','number_of_parts']].drop_duplicates(subset='set_num')
grouped_minifigs = data_minifigs.groupby('set_num')['minifig_quantitiy'].sum().reset_index()        # sum minifig_quantity per set
grouped_colors = data_parts.groupby(['set_num'])['color_id'].nunique().reset_index()  
grouped_categories = data_parts.groupby(['set_num'])['category_id'].nunique().reset_index()         # group by 'set_num' and 'year' and calculate the mean number of unique categories
data_unique_parts = data_parts[data_parts['part_quantity'] == 1]                                    # filter for parts with quantity 1
grouped_unique_parts = data_unique_parts.groupby(['set_num'])['part_num'].nunique().reset_index()
grouped_different_parts = data_parts.groupby(['set_num']).size().reset_index(name='count')

# merge data
merged = pd.merge(grouped_years, grouped_themes, on='set_num', how='left')
merged = pd.merge(merged, grouped_parts, on='set_num', how='left')
merged = pd.merge(merged, grouped_minifigs, on='set_num', how='left').fillna(0)
merged = pd.merge(merged, grouped_colors, on='set_num', how='left')
merged = pd.merge(merged, grouped_categories, on='set_num', how='left')
merged = pd.merge(merged, grouped_unique_parts, on='set_num', how='left')
merged = pd.merge(merged, grouped_different_parts, on='set_num', how='left')
merged.rename(columns={'color_id': 'number_of_colors', 'category_id': 'number_of_categories', 'id': 'category_id', 'part_num': 'number_of_unique_parts', 'count': 'number_of_different_parts'}, inplace=True)

# calculate the proportion for each set
merged['not_unique_parts'] = merged['number_of_parts'] - merged['number_of_unique_parts']
merged['proportion_unique_to_not_unique'] = np.divide(merged['number_of_unique_parts'], merged['not_unique_parts'], out=np.zeros_like(merged['number_of_unique_parts']), where=merged['not_unique_parts']!=0)

merged.to_csv('../Prepared_Data/data.csv', index=False)
data = merged.copy()

# Calculate Complexity with...:

* ...number_of_parts: More parts usually mean more complexity.
* ...number_of_colors: More colors can increase the difficulty of finding the right piece.
* ...number_of_categories: More categories of parts can increase the complexity.
* ...number_of_unique_parts: Unique parts may require specific placement, increasing complexity.
* ...number_of_different_parts: Different parts increase the variety of assembly methods, increasing complexity.

In [11]:
##### Calculate Complexity for each set #####

def calculate_complexity(df):

    # calculate complexity
    df['complexity_score'] = df['number_of_parts'] + df['number_of_colors'] + df['number_of_categories'] + (df['number_of_unique_parts']) + (df['number_of_different_parts']) + df['minifig_quantitiy']
    # normalize complexity
    df['complexity_score'] = (df['complexity_score'] - df['complexity_score'].min()) / (df['complexity_score'].max() - df['complexity_score'].min()) 
    return df

# call function
data_complexity = calculate_complexity(data)
data_complexity.to_csv('../Prepared_Data/data.csv', index=False)